In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'name': ['A', 'A', 'B', 'B', 'B'],
    'v': [3, 4, 2, 4, 3]
})

In [ ]:
df

In [ ]:
df.groupby('name')['v'].transform('mean')

In [2]:
structures = pd.read_feather('../data/structures.feather')

In [11]:
labelled = pd.read_feather('../data/train.feather')

In [3]:
for d in ['x', 'y', 'z']:
    structures[f'{d}_mean'] = structures.groupby('molecule_name')[d].transform("mean")

In [4]:
structures['dist_to_mean'] = pd.Series(np.linalg.norm(structures[['x', 'y', 'z']].values - structures[['x_mean', 'y_mean', 'z_mean']].values, axis=1), index=structures.index)

In [14]:
structures.head(40)

,molecule_name,atom_index,atom,x,y,z,x_mean,y_mean,z_mean,dist_to_mean
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,-0.012689,1.085797,0.008001,0.000012
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,-0.012689,1.085797,0.008001,1.091945
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,-0.012689,1.085797,0.008001,1.091946
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,-0.012689,1.085797,0.008001,1.091954
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,-0.012689,1.085797,0.008001,1.091954
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564,0.093086,0.934733,-0.192278,0.301260
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377,0.093086,0.934733,-0.192278,0.939879
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758,0.093086,0.934733,-0.192278,0.939876
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543,0.093086,0.934733,-0.192278,0.939962
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602,0.300732,0.766301,0.003276,0.396141


In [15]:
structures.to_feather('../data/structures_enh.feather')